# Deploy model

In [114]:
%reload_ext autoreload
%autoreload 2

## Setup

In [115]:
import os
import shutil
from fastai.vision import *
import sys
sys.path.append('../scripts/')
import mfc_functions

In [116]:
kMY_WAVS_DIR = './my_wavs'
kMY_MFCS_DIR = './my_mfcs'
path_img = '../mfc_dataset'  

# export.pkl is stored together with the training + validation data inside path_img.
# export.pkl is generated in `03_train_model.ipynb` by calling learn.export().

In case you want to delete the MFC JPEGs generated by me:

In [117]:
# shutil.rmtree(kMY_MFCS_DIR)
# os.makedirs(kMY_MFCS_DIR)

## Convert WAVs in `./my_wavs` to MFC JPEGs in `./my_mfcs`

In [118]:
mfc_functions.kDEBUG = True  # print out the length and the array for each WAV for error spotting

In [119]:
for wav_fname in filter(lambda wav_fname : wav_fname.split('.')[-1] == 'wav', os.listdir(kMY_WAVS_DIR)):
    mfc_functions.pipeline(
        wav_fpath=os.path.join(kMY_WAVS_DIR, wav_fname), 
        save_fpath=os.path.join(kMY_MFCS_DIR, wav_fname.split('.')[0] + '.jpg')
    )

(47520,) [-0.       -0.       -0.       -0.       ...  0.000679  0.000429  0.000265  0.000123]
(47520,) [-0.000000e+00 -0.000000e+00 -0.000000e+00 -0.000000e+00 ...  1.111098e-04  8.048130e-05  5.873881e-05  2.234124e-05]
(47520,) [-0.       -0.       -0.       -0.       ...  0.000351  0.00036   0.000391  0.00041 ]
(47520,) [-0.000000e+00 -0.000000e+00 -0.000000e+00 -0.000000e+00 ... -2.933226e-05 -3.836700e-05 -4.102544e-05 -1.479598e-05]


## Instantiate a Inference Learner

In [120]:
learn = load_learner(path_img, test=ImageList.from_folder(kMY_MFCS_DIR))

In [121]:
learn.data.test_ds.items

array([PosixPath('my_mfcs/me_seven_male.jpg'), PosixPath('my_mfcs/mywife_six_maleLike.jpg'),
       PosixPath('my_mfcs/mywife_one_maleLike.jpg'), PosixPath('my_mfcs/me_nine_femaleLike.jpg')], dtype=object)

## Inference

In [134]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [123]:
i2c = { v : k for k, v in learn.data.c2i.items() }

In [136]:
print('{:40s} {:15s}'.format('File Path', 'Classification'))
print(''.join(['='] * 55))
for mfc_fpath, y in zip(learn.data.test_ds.items, preds.argmax(axis=1).numpy()):
    print('{:40s} {:15s}'.format(str(mfc_fpath), i2c[y]))

File Path                                Classification 
my_mfcs/me_seven_male.jpg                male           
my_mfcs/mywife_six_maleLike.jpg          male           
my_mfcs/mywife_one_maleLike.jpg          female         
my_mfcs/me_nine_femaleLike.jpg           female         


For each classification, you can listen to the corresponding WAV below to verify: